<a href="https://colab.research.google.com/github/Aerimmm/STUDY/blob/main/fra_eng_Trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [31]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path , filename)
with http.request("GET", url, preload_content = False) as r, open(zipfilename, 'wb') as out_file :
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref :
  zip_ref.extractall(path)

In [32]:
lines = pd.read_csv('fra.txt', names = ['src','tar','lic'], sep = '\t')
del lines['lic']
print("전체 샘플의 개수 : ", len(lines))

전체 샘플의 개수 :  197463


In [33]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 60000개만 저장
lines.sample(10)

,src,tar
28234,Your house is big.,Ta maison est grande.
53791,What's all this noise?,Quel est tout ce bruit ?
54514,You're perfect for me.,Vous êtes parfaite pour moi.
48008,Did you say something?,Disais-tu quelque chose ?
59112,Something wasn't right.,Quelque chose clochait.
54478,You're my kind of gal.,T'es mon type de gonzesse.
58710,My wife is a good cook.,Ma femme est bonne cuisinière.
45722,Tom can speak French.,Tom peut parler français.
48715,He ignored her advice.,Il ignora son conseil.
16749,"We're here, too.",Nous aussi nous sommes ici.


In [34]:
lines.tar = lines.tar.apply(lambda x: '\t' + x + '\n')
lines.sample(10)

,src,tar
536,Get lost.,\tDéguerpissez.\n
1137,I'll obey.,\tJ'obéirai.\n
14869,I'm teasing you.,\tJe te taquine.\n
54874,Beef is very expensive.,\tLe bœuf est très cher.\n
57639,I'll study your report.,\tJe vais étudier ton rapport.\n
28942,Give me more money.,\tDonne-moi plus d'argent.\n
34101,You're out of luck.,\tVotre chance est épuisée.\n
38221,That's nice to know.,\tC'est chouette de le savoir.\n
43515,I'm all out of ideas.,\tJe suis dépourvu d'idées.\n
21319,Tom was cheerful.,\tTom était de bonne humeur.\n


In [35]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src : # 1줄씩 읽음
  for char in line : #1개의 문자씩 읽음
    src_vocab.add(char)


tar_vocab = set()
for line in lines.tar :
  for char in line :
    tar_vocab.add(char)

In [36]:
# 문자 집합의 크기 보기
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print('source 문장의 char 집합 : ', src_vocab_size)
print('target 문장의 char 집합 : ', tar_vocab_size)

source 문장의 char 집합 :  79
target 문장의 char 집합 :  105


In [37]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [38]:
# 각 문자에 인덱스 부여
src_to_index = dict([word, i+1] for i , word in enumerate(src_vocab))
tar_to_index = dict([word, i+1] for i , word in enumerate(tar_vocab))
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [39]:
# 인덱스가 부여된 문자 집합으로부터 갖고있는 훈련 데이터에 정수 인코딩 진행.

encoder_input = []

# 1개의 문장
for line in lines.src :
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line :
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print("source 문장의 정수 인코딩 : ", encoder_input[:5])

source 문장의 정수 인코딩 :  [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [40]:
# 디코더의 입력이 될 프랑스어 데이터에 대해서 정수 인코딩 진행
decoder_input = []
for line in lines.tar :
  encoded_line = []
  for char in line :
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print("target 문장의 정수 인코딩 : ", decoder_input[:5])

target 문장의 정수 인코딩 :  [[1, 48, 53, 3, 4, 2], [1, 39, 53, 70, 55, 60, 57, 14, 2], [1, 31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 2], [1, 28, 67, 73, 59, 57, 3, 4, 2], [1, 45, 53, 64, 73, 72, 3, 4, 2]]


In [41]:
# 정수 인코딩  과정에서 <sos> 제거 \t 제거

decoder_target = []
for line in lines.tar :
  timestep = 0
  encoded_line = []
  for char in line :
    if timestep > 0 :
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 : ', decoder_target[:5])

target 문장 레이블의 정수 인코딩 :  [[48, 53, 3, 4, 2], [39, 53, 70, 55, 60, 57, 14, 2], [31, 66, 3, 70, 67, 73, 72, 57, 3, 4, 2], [28, 67, 73, 59, 57, 3, 4, 2], [45, 53, 64, 73, 72, 3, 4, 2]]


In [42]:
# Padding 작업
max_src_len  = max([len(line) for line in lines.src])
max_tar_len  = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 : ', max_src_len)
print('target 문장의 최대 길이 : ', max_tar_len)

source 문장의 최대 길이 :  23
target 문장의 최대 길이 :  74


In [43]:
# 영어 데이터의 샘플은 23으로 패딩, 프랑스 데이터 샘플은 74로 패딩
encoder_input = pad_sequences(encoder_input, maxlen = max_src_len , padding = 'post')
decoder_input = pad_sequences(decoder_input, maxlen = max_tar_len, padding = 'post')
decoder_target = pad_sequences(decoder_target, maxlen = max_tar_len, padding = 'post')

In [44]:
# 모든 값에 대해 원핫인코딩 수행
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [45]:
# 교사 강요 (teacher forcing)
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [46]:
encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(units = 256, return_state = True)

# encoder_outputss 는 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM 은 바닐라 RNN 과는 달리 상태가 두 개, 은닉상태와 셀 상태
encoder_states = [state_h, state_c] # 이게 Context Vector 이다.

In [47]:
decoder_inputs = Input(shape  = (None, tar_vocab_size))
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state = True)

# 디코더에게 인코더의 은닉, 셀 상태 전달
decoder_outputs , _, _ = decoder_lstm(decoder_inputs, initial_state=  encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation =  'softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

In [48]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size = 64, epochs = 10, validation_split = 0.2)

Epoch 1/10
750/750 [==============================] - 325s 425ms/step - loss: 0.7549 - val_loss: 0.6796
Epoch 2/10
750/750 [==============================] - 316s 421ms/step - loss: 0.4637 - val_loss: 0.5443
Epoch 3/10
750/750 [==============================] - 317s 423ms/step - loss: 0.3885 - val_loss: 0.4859
Epoch 4/10
750/750 [==============================] - 316s 422ms/step - loss: 0.3459 - val_loss: 0.4466
Epoch 5/10
750/750 [==============================] - 314s 419ms/step - loss: 0.3172 - val_loss: 0.4207
Epoch 6/10
750/750 [==============================] - 312s 416ms/step - loss: 0.2961 - val_loss: 0.4038
Epoch 7/10
750/750 [==============================] - 316s 422ms/step - loss: 0.2799 - val_loss: 0.3916
Epoch 8/10
750/750 [==============================] - 312s 416ms/step - loss: 0.2667 - val_loss: 0.3817
Epoch 9/10
750/750 [==============================] - 311s 414ms/step - loss: 0.2558 - val_loss: 0.3751
Epoch 10/10
750/750 [==============================] - 311s 414m

In [49]:
# 번역기 동작시키기
encoder_model = Model(inputs = encoder_inputs, outputs= encoder_states)

In [51]:
#  이전 시점들의 상태들을 저장하는 Tensor
decoder_state_input_h = Input(shape = (256, ))
decoder_state_input_c = Input(shape = (256, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] 

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state) 를 이전 시점의 상태로 사용,. 
# 뒤의 함수 decode_sequence() 에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states= [state_h, state_c]
decoder_outputss = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs= [decoder_inputs] + decoder_states_inputs, outputs = [decoder_outputs] + decoder_states )

In [52]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [62]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = " "

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence
  return decoded_sentence

In [63]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 24ms/step


KeyError: ignored